In [ ]:
# Notebook: RAG with LangChain FAISS OpenAI
# Author: Thomas Purk
# Date: 2025-04-15
# Reference: https://www.gutenberg.org/ebooks/23666
# Reference: https://python.langchain.com/docs/integrations/vectorstores/faiss/

# RAG with LangChain OpenAI

- Retrieval-Augmented Generation (RAG)
- Load documents
- Embed document text
- Index documents using FAISS
- Retrieve relevant chunks based on user query
- Generate a response using OpenAI GPT (3.5/4)

**Token Access Required**

This Google Colab Notebook uses the built in Colab Secrets feature to supply
access to the following tokens
- Hugging Face
- OpenAI

**Limitation**
- Difficulty setting up faiss-gpu in google colab, using faiss-cpu instead.
- If GPU-based sentence embeddings is needed, then the sentence-transformer package has worked in other notebooks and could be used here.

## Notebook Setup

In [9]:
#!pip install langchain-community unstructured faiss-cpu langchain-openai langchain-huggingface

In [1]:
!pip list | grep "langchain\|langchain-community\|unstructured\|faiss-cpu\|langchain-openai\|langchain-huggingface"

faiss-cpu                             1.10.0
langchain                             0.3.23
langchain-community                   0.3.21
langchain-core                        0.3.53
langchain-huggingface                 0.1.2
langchain-openai                      0.3.14
langchain-text-splitters              0.3.8
unstructured                          0.17.2
unstructured-client                   0.32.3


In [5]:
# Setup the Notebook

# General
from google.colab import userdata
import os
import json
import logging
logging.getLogger("transformers").setLevel(logging.WARNING) # Suppress unnecessary logging

# Visualization
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt

# Data, Science, & Math
import numpy as np
import pandas as pd

# Machine/Deep Learning
import torch

# NLP
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.base import BaseCallbackHandler

# Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Arrange Data

In [6]:
# Load a Public Domain woodworking book
# "Mission Furniture: How to Make It, Part 3 by H. H. Windsor"
url_loader = UnstructuredURLLoader(["https://www.gutenberg.org/cache/epub/23666/pg23666.txt"])
documents = url_loader.load()

# Since we passed a single URL we get a single document
print(f'Loaded {len(documents)} document')

Loaded 1 document


In [7]:
# Split into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
docs = text_splitter.split_documents(documents)

# Remove the first 3 documents, they contain Project Guttenberg boilerplate
docs = docs[3:]
# Additional boilerplate at the end is removed also
docs = docs[:307]

print(f'{len(documents)} Documents split into {len(docs)} chunks')

1 Documents split into 307 chunks


## Create Sentence-based Text Embeddings

In [10]:
# Load the model for creating embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Index the document embeddings with FAISS
vectorstore = FAISS.from_documents(docs, embedding_model)

## Define the Model

In [12]:
# Create a callback so we can print the final prompt that the RAG
# system send to OpenAI based on our questions
class PromptPrinter(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"\n{'=' * 50}\nPrompt sent to LLM:\n")
        for prompt in prompts:
            print(prompt)
            print("—" * 50)
        print(f"{'=' * 50}")

# Define the search retriever
vs_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 4, # Number of documents to return
        }
    )

# Initialize OpenAI LLM
openai_llm = ChatOpenAI(
    temperature=0.2,
    model_name="gpt-4o-mini",
    callbacks=[PromptPrinter()]
)


# Create a Retrieval-Augmented QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=vs_retriever,
    return_source_documents=True
)

# Execute - Ask Some Questions

In [13]:
# General Question
query = "What are the key points discussed in the document?"
result = qa_chain.invoke({"query": query})


Prompt sent to LLM:

System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
[Illustration: Detail of the Card Table]

[Illustration: Card Table Ready for Use]

There only remains to fit in the shelves and fasten the top and back. The top and back are held with screws as shown in sketch.

Taper the keys only slightly, otherwise they will keep working loose.

Stain with two coats of weathered oak, give one coat of thin shellac to fix the stain and two coats of wax for a soft-gloss finish.

A WRITING DESK

Sections Detail of the Bookrack The Complete Bookrack Detail of the Table Table for the Dining-Room Set Armchair of the Dining-Room Set Detail of the Armchair Detail of the Hall Bench Bench Made of Plain Oak Sewing Table in Plain Oak Detail of the Sewing Table Construction of the Drawer Side Chair of Dining-Room Set Detail of the Side Chair Detail of the Pia

In [14]:
print("Question:")
print(query)
print("\n Answer:")
display(result["result"])

Question:
What are the key points discussed in the document?

 Answer:


'The document discusses the construction and finishing details of various furniture pieces, including a card table, writing desk, dining room set, armchair, hall bench, sewing table, side chair, piano bench, magazine rack, and a den table. Key points include:\n\n1. **Assembly Instructions**: Details on fitting shelves, fastening the top and back with screws, and tapering keys slightly to prevent them from loosening.\n2. **Finishing Techniques**: Recommendations for staining with weathered oak, applying thin shellac to fix the stain, and using wax for a soft-gloss finish.\n3. **Illustrations**: Visual details of each furniture piece, highlighting construction and design elements.\n\nOverall, the document provides guidance on building and finishing wooden furniture items.'

In [15]:
# Specific Question
query = "What is the strongest type of joint to use for a chair?"
result = qa_chain.invoke({"query": query})


Prompt sent to LLM:

System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
[Illustration: Arm Chair Complete]

[Illustration: Detail of the Arm Chair]

A mission arm chair of simple design and construction is shown in the accompanying illustration. This chair is suitable for any room of the house and can be made of wood to match other furniture. Quarter-sawed oak is the wood most generally used, and it is also very easy to obtain. The stock can be ordered from the mill, cut to length, squared and sanded. Following is a list of the material that will be needed:

This armchair will look well if made of plain-sawed oak. Quarter-sawed oak might be used, or black walnut if desired. The stock bill specifies the various parts mill-planed to size as far as possible. If some amateur craftsman should prefer to do his own surfacing, thereby saving somewhat on the exp

In [16]:
print("Question:")
print(query)
print("\n Answer:")
display(result["result"])

Question:
What is the strongest type of joint to use for a chair?

 Answer:


"I don't know."

In [17]:
# Specific Question 2
query = "What is the best wood to use for making a table?"
result = qa_chain.invoke({"query": query})


Prompt sent to LLM:

System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The accompanying cut shows a magazine rack that will find favor with many amateur wood-workers on account of its simplicity in design and its rich, massive appearance when properly finished. It is so constructed that each piece may be polished, stained and finished before it is finally put together. Quarter-sawed oak is the best wood to use. Plain-sawed oak looks well, but it is more liable to warp than quarter-sawed and this is quite an element in pieces as wide as the ones here used.

A PLATE RACK FOR THE DINING ROOM

This plate rack can be made of any kind of wood and finished to match other pieces of furniture in the room, but as it is of mission design, oak is the most suitable lumber, as it takes the mission stain so nicely.

The material required is as follows:

A WALL SHELF


In [18]:
print("Question:")
print(query)
print("\n Answer:")
display(result["result"])

Question:
What is the best wood to use for making a table?

 Answer:


'The best wood to use for making a table, especially in mission style, is quarter-sawed oak. It is recommended for its durability and ability to take a stain nicely.'

In [19]:
result

{'query': 'What is the best wood to use for making a table?',
 'result': 'The best wood to use for making a table, especially in mission style, is quarter-sawed oak. It is recommended for its durability and ability to take a stain nicely.',
 'source_documents': [Document(id='e5ad7e2b-5acd-4135-97a2-11179b02077b', metadata={'source': 'https://www.gutenberg.org/cache/epub/23666/pg23666.txt'}, page_content='The accompanying cut shows a magazine rack that will find favor with many amateur wood-workers on account of its simplicity in design and its rich, massive appearance when properly finished. It is so constructed that each piece may be polished, stained and finished before it is finally put together. Quarter-sawed oak is the best wood to use. Plain-sawed oak looks well, but it is more liable to warp than quarter-sawed and this is quite an element in pieces as wide as the ones here used.'),
  Document(id='141ff81c-7598-4001-9ab0-6fa68f779b82', metadata={'source': 'https://www.gutenberg.o